In [1]:
# Import necessary modules

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import geopandas as gpd
from urllib.request import urlretrieve
import requests
from glob import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
from datetime import datetime, timedelta
import json
import sys
import math
from geopy.distance import distance
from matplotlib.colors import to_rgba
from collections import defaultdict

In [2]:
# Establish directory locations

parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/sat_plot')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')
floats_dir    = os.path.join(parent_dir, 'Data/Floats')

In [3]:
# Plotting preferences

# Global

start_date = datetime(2024, 5, 20) #YYYY, M, DD
end_date = datetime(2024, 5, 30)

min_lon = -35
max_lon = -5
min_lat = 55
max_lat = 66

# Float colors
# For additional floats, add the name (as it appears in the 'Float_positions.csv') and color you'd like (as a hexcode)
float_colors = {
        '4903532':  '#B4184C',
        'navis102': '#F5A300',
        '1902637':  '#0000E0',
        'navis101': '#FBFF1F'}

# CHLA
skip_chla            = True
CHLA_color           = 'YlGnBu_r'
CHLA_plot_as_log     = True # Default True
CHLA_plot_lim_max    = 10   # Default 10
CHLA_plot_lim_min    = 0.1  # Default 0.1

# SST
skip_sst             = True
SST_color            = 'YlOrRd'
SST_plot_as_log      = False # Default False
SST_plot_lim_max     = 15    # Default 15
SST_plot_lim_min     = 5     # Default 5

# SSH
skip_sla             = False
SSH_color            = 'RdBu'
SSH_plot_as_log      = False # Default False
SSH_plot_lim_max     = 0.5   # Default  0.5
SSH_plot_lim_min     = -0.5  # Default -0.5

# PIC
skip_pic             = False
PIC_color            = 'YlGnBu_r'
PIC_plot_as_log      = True
PIC_plot_lim_max     = 0.1
PIC_plot_lim_min     = 0.0000001

In [4]:
# Function to generate points on a circle centered at (lat, lon) with a given radius
def generate_circle_points(lat, lon, radius_km, num_points=360):
    angles = np.linspace(0, 360, num_points)
    circle_points = []
    for angle in angles:
        point = distance(kilometers=radius_km).destination((lat, lon), angle)
        circle_points.append((point.latitude, point.longitude))
    
    return np.array(circle_points)

# Generates a list of dates between given dates 
def get_dates_between(start_date, end_date):
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    return dates

In [5]:
### Plotting most recent data from CMEMS
## List most recent files

# List to store matching file names
matching_files = []

# Loop through the files in the directory
for filename in os.listdir(NEODASS_dir):
    if filename.endswith('.nc') and f'-1d-' in filename:
        matching_files.append(filename)

# Dictionary to store files by their base names (excluding '_a_' or '_b_') and key variables
file_dict = {}

for file in matching_files:
    # Identify the base name by removing '_a' or '_b' if present
    base_name = file.replace('_a', '_').replace('_b', '_')
    if base_name not in file_dict:
        file_dict[base_name] = {}
    filepath = os.path.join(NEODASS_dir, file)
    ds = xr.open_dataset(filepath)
    variables = list(ds.data_vars.keys())
    #print(variables)

    # Find the key variable excluding 'longitude' and 'latitude'
    key_variables = [var for var in variables if 'longitude' not in var.lower() and 'latitude' not in var.lower()]

    if key_variables:
        time_str = ds['time'].dt.strftime("%Y-%m-%d").item()
        for key_variable in key_variables:
            file_dict[base_name][file] = {
                'variable': key_variable,
                'date': datetime.strptime(time_str, "%Y-%m-%d")
                }       
    else:
        print(f"No suitable key variable found in file: {file}")

    ds.close()

# Group files by variable category and day
var_dict = {'chl': {}, 'temp': {}, 'sla': {}, 'adt': {}, 'bbp': {}, 'pic': {}}

for base_name, files in file_dict.items():
    for file, data_info in files.items():
        data_var = data_info['variable']
        date = data_info['date']
        var_name_lower = data_var.lower()
        #print(data_var)
        if 'chl' in var_name_lower:
            if date not in var_dict['chl']:
                var_dict['chl'][date] = []
            var_dict['chl'][date].append((file, data_var))
        elif 'temp' in var_name_lower:
            if date not in var_dict['temp']:
                var_dict['temp'][date] = []
            var_dict['temp'][date].append((file, data_var))
        elif 'sla' in var_name_lower:
            if date not in var_dict['sla']:
                var_dict['sla'][date] = []
            var_dict['sla'][date].append((file, data_var))
        elif 'adt' in var_name_lower:
            if date not in var_dict['adt']:
                var_dict['adt'][date] = []
            var_dict['adt'][date].append((file, data_var))
        elif 'bbp' in var_name_lower:
            if date not in var_dict['bbp']:
                var_dict['bbp'][date] = []
            var_dict['bbp'][date].append((file, data_var))
        elif 'pic' in var_name_lower:
            if date not in var_dict['pic']:
                var_dict['pic'][date] = []
            var_dict['pic'][date].append((file, data_var))

## Print the results
for var_category, var_data in var_dict.items():
    print(f"{var_category.capitalize()} files:")
    for date, files in var_data.items():
        print(f"  Date: {date}")
        for file, variable in files:
            print(f"    File: {file}, Variable: {variable}")

# We load shape files for bathymetry lines
#first_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_J_1000.shp'
#second_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_I_2000.shp'

first_line_path  = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_J_1000.shp'
second_line_path = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_I_2000.shp'

gdf_1000 = gpd.read_file(first_line_path)
gdf_2000 = gpd.read_file(second_line_path)

Chl files:
  Date: 2024-05-20 00:00:00
    File: olci_a_1km-CHL_OC4ME-1d-20240520_20240520-nrt.nc, Variable: CHL_OC4ME
    File: olci_b_1km-CHL_OC4ME-1d-20240520_20240520-nrt.nc, Variable: CHL_OC4ME
    File: viirs_noaa20-chl_oc5ci-1d-20240520_20240520-nrt.nc, Variable: chl_oc5ci
  Date: 2024-05-21 00:00:00
    File: olci_a_1km-CHL_OC4ME-1d-20240521_20240521-nrt.nc, Variable: CHL_OC4ME
    File: olci_b_1km-CHL_OC4ME-1d-20240521_20240521-nrt.nc, Variable: CHL_OC4ME
    File: viirs_noaa20-chl_oc5ci-1d-20240521_20240521-nrt.nc, Variable: chl_oc5ci
  Date: 2024-05-22 00:00:00
    File: olci_a_1km-CHL_OC4ME-1d-20240522_20240522-nrt.nc, Variable: CHL_OC4ME
    File: olci_b_1km-CHL_OC4ME-1d-20240522_20240522-nrt.nc, Variable: CHL_OC4ME
    File: viirs_noaa20-chl_oc5ci-1d-20240522_20240522-nrt.nc, Variable: chl_oc5ci
  Date: 2024-05-23 00:00:00
    File: olci_a_1km-CHL_OC4ME-1d-20240523_20240523-nrt.nc, Variable: CHL_OC4ME
    File: olci_b_1km-CHL_OC4ME-1d-20240523_20240523-nrt.nc, Variable: C

In [6]:
# Plotting loop

zoom_plot = False

csv_file = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\Plotting_tools\\shared_data\\rt_positions.csv'  # Update this path
df = pd.read_csv(csv_file)

# 130km circle around deploy point
circle_points = generate_circle_points(60, -24, 444)
circle_lats, circle_lons = circle_points[:, 0], circle_points[:, 1]

# Load float positions
float_position_path = os.path.join(floats_dir, 'Float_positions_2024_05_25.csv')
full_position = pd.read_csv(float_position_path)

# Convert the 'time' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Define the date_of_plot as a string in the format 'YYYY-MM-DD'
date_of_plot = '2024-05-30'  # Example date
date_of_plot_dt = datetime.strptime(date_of_plot, '%Y-%m-%d')

# Define unique symbols for different object types
object_symbols = {
    'glider': 'o',
    'Float': 's',
    'Ship'  : '^'
    # Add more object types and their symbols here
}

# Define colors for different objects
object_colors = {
    'unit_345': '#53599A',  # Example color
    'unit_397': '#7A7FB8',
    'unit_398': '#AFB3D4',
    'unit_405': '#32365D',
    'lovuse026d': '#F7A072',  # Example color
    'lovuse031c': '#F47E3E',  # Example color
    'lovuse032c': '#D4520C',  # Example color
    'Discovery' : '#70163C'
    # Add more object names and their colors here
}
# Date strings for plotting
dates_to_plot = get_dates_between(start_date, end_date)

for var_category, files in var_dict.items():

    if skip_chla and var_category == 'chl':
        continue
    if skip_sst and var_category == 'temp':
        continue
    if skip_sla and var_category == 'sla':
        continue
    if skip_pic and var_category == 'pic':
        continue

    for date, files_per_day in tqdm(files.items(), desc='Date progress...'):  # Iterate over each day

        if date.strftime("%Y-%m-%d") not in dates_to_plot:
            continue

        daily_data = []  # List to store data variables

        # Collect all data variables for the day
        for file, data_var in files_per_day:
            try:
                print(f"Opening file: {file} for variable: {data_var}")
                cur_data_var = xr.open_dataset(os.path.join(NEODASS_dir, file))[data_var]
                daily_data.append(cur_data_var)
                print(f"Appended data for {data_var} from file: {file}")
            except Exception as e:
                print(f"Error processing file: {file} for variable: {data_var}. Error: {e}")

        # Align all datasets spatially
        aligned_data = xr.align(*daily_data, join='outer') 
        
        # Combine aligned data into a single dataset
        combined_data = xr.concat(aligned_data, dim='time').mean(dim='time', skipna=True)

        print(f"Averaged data for {var_category} on {date}:")
        # Print or process combined_data as needed

        # Format the date for plot title
        date_of_plot = date.strftime("%Y-%m-%d")
        
        #print(*daily_data, sep='\n')
        #print(combined_data, sep='\n')


        if var_category == 'chl':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'Chlorophyll (' + xx_plot_units + ')'
            xx_plot_min = CHLA_plot_lim_min
            xx_plot_max = CHLA_plot_lim_max
            color = CHLA_color
            log_scaling = CHLA_plot_as_log
            xx_output_dir_name = 'Chla'

        elif var_category == 'temp':
            xx_plot_units = cur_data_var.attrs.get('units')
            if xx_plot_units == 'kelvin':
                # Convert Kelvin to Celsius
                combined_data = combined_data - 273.15
                degree_sign = u'\N{DEGREE SIGN}'
                xx_plot_cbar_label = 'Sea Surface Temperature (' + degree_sign + 'C)'
                xx_plot_min = SST_plot_lim_min
                xx_plot_max = SST_plot_lim_max
                color = SST_color
                log_scaling = SST_plot_as_log
                xx_output_dir_name = 'SST'
                
        elif var_category == 'sla':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'Sea level anomaly (' + xx_plot_units + ')'
            xx_plot_min = SSH_plot_lim_min
            xx_plot_max = SSH_plot_lim_max
            color = SSH_color
            log_scaling = SSH_plot_as_log
            xx_output_dir_name = 'SLA'

        elif var_category == 'adt':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'Sea surface velocity (' + xx_plot_units + 's$^{-1}$)'
            xx_plot_min = SSH_plot_lim_min
            xx_plot_max = SSH_plot_lim_max
            color = SSH_color
            log_scaling = SSH_plot_as_log
            xx_output_dir_name = 'SSC'

        elif var_category == 'bbp':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'Particulate backscatter (' + xx_plot_units + ')'
            xx_plot_min = BBP_plot_lim_min
            xx_plot_max = BBP_plot_lim_max
            color = BBP_color
            log_scaling = BBP_plot_as_log
            xx_output_dir_name = 'BBP'

        elif var_category == 'pic':
            xx_plot_units = cur_data_var.attrs.get('units')
            xx_plot_cbar_label = 'PIC (' + xx_plot_units + ')'
            xx_plot_min = PIC_plot_lim_min
            xx_plot_max = PIC_plot_lim_max
            color = PIC_color
            log_scaling = PIC_plot_as_log
            xx_output_dir_name = 'PIC'

        # Dependant settings based on each variable type
        else:
            print(f"No settings set for '{var_category}', skipping!")
            continue
        
        norm = colors.LogNorm(vmin=xx_plot_min, vmax=xx_plot_max) if log_scaling else colors.Normalize(vmin=xx_plot_min, vmax=xx_plot_max) # Setting color scale based on Log options.
        #print(combined_data.max())
        #print(combined_data.min())
        # Plotting

        combined_data = np.clip(combined_data, xx_plot_min, xx_plot_max)
        fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
        ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
        im = ax.pcolormesh(combined_data['longitude'], combined_data['latitude'], 
                            combined_data, 
                            cmap=color, 
                            norm=norm, 
                            transform=ccrs.PlateCarree())
        
        cbar = plt.colorbar(im, ax=ax, label=xx_plot_cbar_label)

        legend_handles = []
        # Process each unique object type
        # Filter out rows with 'time' greater than date_of_plot

        df_filtered = df[df['date'] <= date_of_plot]

        for obj_type, symbol in object_symbols.items():

            if obj_type == 'ship':
                continue
            if obj_type == 'glider':
                continue

            # Filter data for the current object type
            type_data = df_filtered[df_filtered['platform_type'] == obj_type]
            unique_names = type_data['platform_id'].unique()
            
            for name in unique_names:

                obj_data = type_data[type_data['platform_id'] == name].sort_values(by='date', ascending=False)  # Sort in descending order
                
                # Get the colors and opacities for the points
                rgba_color = to_rgba(object_colors.get(name, '#000000'))  # Default to black if not found
                num_positions = len(obj_data)
                alphas = np.linspace(1, 0.1, min(num_positions, 10))  # Reverse the opacity gradient
        
                # Plot the positions and lines, with the first 10 points having a decreasing alpha
                for i, (index, row) in enumerate(obj_data.iterrows()):
                    if i >= 10:  # Only plot up to the 10 most recent points
                        break
                    alpha = alphas[i]
                    rgba_color_with_alpha = (*rgba_color[:3], alpha)
                    marker_edge_color = (*rgba_color[:3], alpha)  # Marker edge color follows opacity gradient
                    ax.scatter(row['lon'], row['lat'], color=rgba_color_with_alpha, edgecolor=marker_edge_color, 
                               transform=ccrs.PlateCarree(), zorder=3, marker=symbol, s=100)
        
                    if i > 0:
                        prev_row = obj_data.iloc[i - 1]
                        ax.plot([prev_row['lon'], row['lon']],
                                [prev_row['lat'], row['lat']],
                                color=rgba_color_with_alpha, transform=ccrs.PlateCarree(), zorder=2)
        
                # Create a proxy artist for each object
                proxy = plt.Line2D([0], [0], linestyle='-', marker=symbol, color=object_colors.get(name, '#000000'), label=name)
                legend_handles.append(proxy)
    
        # Deploy point position marker
        sc3 = ax.scatter(-24,60,transform=ccrs.PlateCarree(), edgecolors='black', facecolors='none', marker='s')
        ax.plot(circle_lons, circle_lats, transform=ccrs.PlateCarree(), color='black')
        
        gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
        gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

        # Add coastlines
        ax.add_feature(cfeature.COASTLINE)

        # Add lon/lat grid and adjust labels
        gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, crs=ccrs.PlateCarree())
        gl.top_labels = False
        gl.right_labels = False

        # Plot title
        plot_title = f'NEODASS_{xx_output_dir_name} {date_of_plot}'
        ax.set_title(plot_title, fontsize=24)

        # Add legend in
        deploy_point_proxy = plt.Line2D([0], [0], linestyle='none', marker='s', color='black', markersize=10, markerfacecolor='none', label='Deploy Point')
        legend_handles.append(deploy_point_proxy)
        ax.legend(handles=legend_handles, loc='lower left')

        # Save output
        save_dir = os.path.join(output_dir, f'{xx_output_dir_name}{"_log" if log_scaling else ""}_TEST_autonomy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        basename_suffix = base_name[:10]
        filename = f'{xx_output_dir_name}{"_log" if log_scaling else ""}_{date_of_plot}_NEODASS.png'
        plt.savefig(os.path.join(save_dir, filename))
        
        plt.clf()
        plt.close()

Date progress...:   0%|          | 0/10 [00:00<?, ?it/s]

Opening file: cmems_sea_level-sla-1d-20240520_20240520-nrt.nc for variable: sla
Appended data for sla from file: cmems_sea_level-sla-1d-20240520_20240520-nrt.nc
Averaged data for sla on 2024-05-20 00:00:00:


Date progress...:  10%|█         | 1/10 [00:19<02:52, 19.18s/it]

Opening file: cmems_sea_level-sla-1d-20240521_20240521-nrt.nc for variable: sla
Appended data for sla from file: cmems_sea_level-sla-1d-20240521_20240521-nrt.nc
Averaged data for sla on 2024-05-21 00:00:00:


Date progress...:  20%|██        | 2/10 [00:36<02:25, 18.15s/it]

...

Date progress...:  80%|████████  | 8/10 [04:15<01:03, 31.79s/it]

Opening file: viirs_suomi-chl_oc5_pic-1d-20240528_20240528-nrt.nc for variable: pic
Appended data for pic from file: viirs_suomi-chl_oc5_pic-1d-20240528_20240528-nrt.nc
Averaged data for pic on 2024-05-28 00:00:00:


Date progress...:  90%|█████████ | 9/10 [04:46<00:31, 31.68s/it]

Opening file: viirs_suomi-chl_oc5_pic-1d-20240529_20240529-nrt.nc for variable: pic
Appended data for pic from file: viirs_suomi-chl_oc5_pic-1d-20240529_20240529-nrt.nc
Averaged data for pic on 2024-05-29 00:00:00:


Date progress...: 100%|██████████| 10/10 [05:18<00:00, 31.81s/it]
